In [7]:
import sys; sys.path.append('..')
from cadence.imports import *
import cadence as cd
import lltk
C=lltk.load('TedJDH')
t=random.choice([t for t in C.texts() if t.medium=='Fiction'])
line=random.choice([
    ln for ln in t.txt.split('\n') if ln.strip() and len(ln)>20
])#.split(',')[0]
# t,line
# line= 'finding that he had ample time, sat down'
#line='mercy and justice chained into the unborn world'

In [12]:
# def process(q, iolock):
#     from time import sleep
#     while True:
#         stuff = q.get()
#         if stuff is None:
#             break
#         with iolock:
#             print("processing", stuff)
#         sleep(stuff)

# NCORE=2
# q = mp.Queue(maxsize=NCORE)
# iolock = mp.Lock()
# pool = mp.Pool(NCORE, initializer=process, initargs=(q, iolock))
# for stuff in range(20):
#     q.put(stuff)  # blocks until q below its max size
#     with iolock:
#         print("queued", stuff)
# for _ in range(NCORE):  # tell workers we're done
#     q.put(None)
# pool.close()
# pool.join()

In [2]:
# txt="""In Xanadu did Kubla Khan
# A stately pleasure-dome decree:
# Where Alph, the sacred river, ran
# Through caverns measureless to man
#    Down to a sunless sea.
# """
# line=txt.split('\n')[0]

In [3]:
# TOTALCOL

In [4]:
poems = C.texts(C.meta.query('medium=="Poetry"'))
# poemss

In [5]:
tnow=C.textd['K046497.000']
tnow

Jones, Henry, 1721-1770, _VERSES TO HIS GRACE THE DUKE OF NEWCASTLE ON THE DEATH OF THE RIGHT HONOURABLE HENRY PELHAM._ (1754) [TedJDH: K046497.000]

In [6]:
tparsed=cd.parse(
    tnow.txt,
    verse=True,
    by_line=True,
    only_best=True,
    num_proc=1
)

Iterating over line scansions [x1]:  55%|█████▍    | 148/271 [00:13<00:08, 14.02it/s]

1


Iterating over line scansions [x1]:  60%|█████▉    | 162/271 [00:15<00:08, 12.42it/s]

1


Metrically parsing lines (+word IPA combinations) [x1]: 100%|██████████| 1/1 [00:00<00:00, 68.03it/s]

...

Metrically parsing lines (+word IPA combinations) [x1]: 100%|██████████| 1/1 [00:00<00:00, 69.22it/s]

1



Metrically parsing lines (+word IPA combinations) [x1]: 100%|██████████| 1/1 [00:00<00:00, 70.06it/s]

1



Iterating over line scansions [x1]: 100%|██████████| 271/271 [00:42<00:00,  6.41it/s]


In [9]:
pd.UndefinedVariableError

AttributeError: module 'pandas' has no attribute 'UndefinedVariableError'

In [14]:
random.shuffle(poems)
for t in poems:
    tparsed=cd.parse(
        t.txt,
        verse=True,
        by_line=True,
        only_best=True
    )
    
#     for linedf in cd.parse_iter(
#             t.txt,
#             verse=True,
#             by_line=True,
#             only_best=False):
#         pass
    break
tparsed

# cd.parse(line,num_proc=1,by_line=True,only_best=True,force=True)

Iterating over line scansions [x1]:   3%|▎         | 8/271 [00:03<01:56,  2.26it/s]

SpecificationError: Column(s) ['combo_num_syll', 'parse_num_pos', 'parse_num_syll'] do not exist

In [8]:
# !rm /home/ryan/.cadence/data/db.lines.sqlite

In [ ]:
PARSELINEKEY

In [ ]:
for x in cd.scan(txt,phrasebreaks=False,num_proc=1): pass

In [ ]:
# for x in cd.parse_iter(
#     txt,
#     phrasebreaks=False,
#     num_proc=1,
#     by_line=True): display(x)
# x

In [ ]:
cd.parse(
    txt,
    phrasebreaks=False,
    num_proc=1,
    by_line=True
)

In [ ]:
# get_num_lines??

In [ ]:
# stop

In [ ]:
!ls -lh /home/ryan/.cadence/data/db.lines.sqlite

In [ ]:
import lltk
C=lltk.load('CanonFiction')
txt_long=C.au.Joyce.Ulysses.txt

In [ ]:
# # %%timeit
# for x in scan_iter(txt_long,verse=False): pass
# x

In [ ]:
!ls -ltrh ~/.cadence/data
# !mvln /home/ryan/.cadence /home/ryan/subspace/.cadence

In [ ]:
len(txt_long)

In [ ]:
# for x in cd.parse_iter(
#     txt_long,
#     prose=True,
#     num_proc=4,
#     by_line=True):
#         #display(x)
#         #break
#         pass
# x

In [ ]:
for linedf in cd.scan_iter(txt): break
len(linedf.drop_duplicates(['word_i','syll_i']))

In [ ]:
dfparses=cd.parse(txt_long[:10000], prose=True, by_line=True)
dfbestparses=dfparses.query('parse_rank==1')
dfbestparses

In [ ]:
dfbestparses.shape, dfparses.shape

In [ ]:
dfbestparses_pentameter = dfbestparses.query('parse_num_syll==10')
dfbestparses_pentameter.sort_values('*total')

In [ ]:
# sby=['stanza_i','line_i','linepart_i']
# dfparses_nocombo = dfparses.reset_index().sort_values(
#     sby+['*total']
# ).drop_duplicates(
#     sby,
#     keep='first'
# )
# setindex(dfparses_nocombo)

In [ ]:
dfparses_nocombo.query('parse_num_syll==10').sort_values('*total',ascending=False)

In [ ]:
dfparses.query(
    'parse_num_syll==10'
)sort_values('*total',ascending=False)

In [ ]:
# !ls -la ~ | grep .cadence

In [ ]:
# for linedf in cd.scan_iter(txt_long,num_proc=1): pass

In [ ]:
# for x in iter_parses(txt_long,progress=False,num_proc=1): pass
# x

In [ ]:
for linedf in cd.scan_iter(txt_long,num_proc=4): pass

In [ ]:
scan(txt)

In [ ]:
cd.iter_combos??

In [ ]:
def

In [ ]:
# dfscan=scan(txt_long,num_proc=4)

In [ ]:
cd.iter_lines??

In [ ]:
dfscan

In [ ]:
for combodf in iter_combos(scan(line)): pass

In [ ]:
odf=parse(line,num_proc=4)

In [ ]:
import lltk
C=lltk.load('CanonFiction')

In [ ]:
dfparses=parse(C.au.Joyce.Ulysses.txt)

In [ ]:
# for

In [ ]:
parse_combo(combodf).reset_index()[pcols]

In [ ]:
# parse_prosodic(combodf) == 

In [ ]:
for combodf in iter_combos(scan(line)): pass
combo_cols=set(combodf.reset_index().columns)
combo_cols
combodf_parsed=parse_combo(combodf).reset_index()
cols_weneed = [c for c in combodf_parsed.columns if c not in combo_cols]
combodf_parsed[cols_weneed]

In [ ]:
combo_parsed_pros = parse_prosodic(combodf)

In [ ]:
combodf_parsed.reset_index()[cols_weneed]

In [ ]:
cols_weneed=set(parse_cols) - set(combo_cols)
cols_weneed

In [ ]:
l=parse_prosodic('Bird bird bird bird', constraints=['*w/peak','*w/stressed','*s/unstressed','*f-res'])

In [ ]:
prs=l.bestParse()
pos=prs.positions[0]
prs.constraintScores

In [ ]:
for posx in prs.positions: print(posx.constraintScores)

In [ ]:
p.config['meters']['default_english'].constraints

In [ ]:
# p.Meter

In [ ]:
def parse_phons():
    output_dict['[*clash_across]']=counts(stress_str,'P#P') + counts(stress_str,'P#S') + counts(stress_str,'S#P') + counts(stress_str,'S#S')
    output_dict['[*clash_within]']=counts(stress_str,'PP') + counts(stress_str,'PS') + counts(stress_str,'SP') + counts(stress_str,'SS')
    output_dict['[*clash_across_primary]']=counts(stress_str,'P#P')
    output_dict['[*clash_within_primary]']=counts(stress_str,'PP')
    output_dict['[*lapse_across]']=counts(stress_str,'U#U')
    output_dict['[*lapse_within]']=counts(stress_str,'UU')
    output_dict['[*WSP]']=0
    output_dict['[*PEAKPROM]']=0
    output_dict['[*High_Stress]']=0
    output_dict['[*Low_Unstress]']=0
    output_dict['[*High_Strong]']=0
    output_dict['[*Low_Weak]']=0
    for s,w,hml,mtr in zip(stress_str,weight_str,sonority_str,meter_str):
        if s=='U' and w=='H':
            output_dict['[*WSP]']+=1
        if (s=='P' or s=='S') and w=='L':
            output_dict['[*PEAKPROM]']+=1
        
        if hml=='H' and s in {'P','S'}:
            output_dict['[*High_Stress]']+=1
        if hml=='L' and s=="U":
            output_dict['[*Low_Unstress]']+=1
        
        if hml=='H' and mtr == 's':
            output_dict['[*High_Strong]']+=1
        if hml=='L' and mtr == 'w':
            output_dict['[*Low_Weak]']+=1